## 2

Die folgenden Werte seien gegeben: 
x -2, -1, 3, 4, 6 
y 0, 0.5, 2, 2, 5 
(a) Stellen Sie das lineare Gleichungssystem Ax = b auf und geben Sie die einzelnen Vektoren und Matrizen an. 
(b) Berechnen Sie die Regressionsgerade. 
(c) Erstellen Sie einen Plot und zeichnen Sie sowohl die Daten, wie die Regressionsgerade und die Residuen ein.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

x = np.array([-2, -1, 3, 4, 6], float)
y = np.array([0, 0.5, 2, 2, 5])
print(f'y:\n{y}')
A = np.vstack([x, np.ones(len(x))]).T
print(f'A:\n{A}')
print(f'A.T @ A:\n{A.T @ A}')

beta = np.linalg.inv(A.T @ A) @ A.T @ y
print(f'beta:\n{beta}')

y_pred = A @ beta
print(f'y predictions:\n{y_pred}')

residuen = y - y_pred
print(f'residuen:\n{residuen}')


plt.plot(x, y_pred, color='red', label='Regressionsgerade')
plt.scatter(x, y, color='blue', label='Daten')
# Horizontale Linie bei y=0
plt.axhline(0, color='black', linewidth=2)
# Vertikale Linie bei x=0
plt.axvline(0, color='black', linewidth=2)
# Residuen als Linien plotten
for xi, yi, yp in zip(x, y, y_pred):
    plt.plot([xi, xi], [yi, yp], color='green', linestyle='--')

# Achsenbeschriftungen und Legende hinzufügen
plt.xlabel('x')
plt.ylabel('y')
plt.title('Lineare Regression und Residuen')
plt.legend()
plt.grid(True)

plt.show()



## 3
Als Data Scientist haben Sie den Auftrag, in einer Firma herauszufinden, welche Mitarbeiter öfters abwesend sind, wovon dies abhängt und wie man Absenzen voraussagen kann. Dazu erhalten Sie den Datensatz employees.csv (Qulle: fiktiver Datensatz von kaggle.com), den Sie vorzugsweise mittels Python analysieren.

 
(a) Daten sind oft nicht bereinigt. Die Firma sagt Ihnen, dass der Praktikant beim Erheben des Datensatzes bestimmt ein paar Tippfehler gemacht hat. Ihnen wird auch gesagt, dass jeder Mitarbeiter mindestens einmal gefehlt hat. Bereinigen Sie den Datensatz.

In [2]:
import pandas as pd
employees = pd.read_csv('data/employees.csv')
employees = employees.loc[employees['AbsentHours'] > 0]



 
(b) Wovon hängen die Absenzen eher ab, von der Anzahl Jahren im Betrieb oder dem Alter der Mitarbeiter? Führen Sie jeweils eine Regression durch.

In [3]:
from sklearn.linear_model import LinearRegression

X_years = employees['LengthService'].values.reshape(-1,1)
X_age = employees['Age'].values.reshape(-1,1)
absent_hours = employees['AbsentHours'].values

years_model = LinearRegression().fit(X_years, absent_hours)
age_model = LinearRegression().fit(X_age, absent_hours)

employees.plot.scatter(x='LengthService', y="AbsentHours")
plt.plot([0,40], years_model.predict(np.array([[0], [40]])), color='r')
plt.show()

employees.plot.scatter(x='Age', y="AbsentHours")
plt.plot([20, 80], age_model.predict(np.array([[20], [80]])), color='r')
plt.show()


(c) Plotten Sie die Residuen mittels Tukey-Anscombe Digrammen. Wie erklären Sie die scharfe Kante im Residuenplot des Alters? 

In [4]:
age_residuals = absent_hours - age_model.predict(X_age)
time_residuals = absent_hours - years_model.predict(X_years)

# Erstellung der Subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6)) # 1 Zeile, 2 Spalten

# Erster Plot für das Alter
ax1.scatter(X_age, age_residuals)
ax1.axhline(0, color='r', linewidth=2)
ax1.set_xlabel('Alter der Mitarbeiter')
ax1.set_ylabel('Residuen der AbsentHours')
ax1.set_title('Tukey-Anscombe Diagramm für das Alter')

# Zweiter Plot für die Anzahl Jahre im Betrieb
ax2.scatter(X_years, time_residuals)
ax2.axhline(0, color='r', linewidth=2)
ax2.set_xlabel('Anzahl Jahren im Betrieb')
ax2.set_ylabel('Residuen der AbsentHours')
ax2.set_title('Tukey-Anscombe Diagramm für die Anzahl Jahren im Betrieb')

# Anzeigen der Plots
plt.tight_layout()
plt.show()



(d) Prüfen Sie, ob die Bedingungen der Residuenanalyse erfüllt sind. 

In [5]:
from scipy.stats import norm

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6)) # 1 Zeile, 2 Spalten


n, bins, patches = ax1.hist(age_residuals, bins=150, facecolor='blue',stacked=True,density=True)
ax1.set_xlabel('Residuen')
ax1.set_ylabel('Haeufigkeit')
ax1.set_title('Sind die Alter Residuen Normalverteilt?')
mu = np.average(age_residuals)
sigma = np.std(age_residuals)
#normalverteilung mit diesem Mittelwert und Standardabweichung
y = norm.pdf(bins, mu, sigma)
ax1.plot(bins, y, 'r--')

n2, bins, patches2 = ax2.hist(time_residuals, bins=150, facecolor='blue',stacked=True,density=True)
ax2.set_xlabel('Residuen')
ax2.set_ylabel('Haeufigkeit')
ax2.set_title('Sind die Residuen der Anzahl Jahre Normalverteilt?')
mu = np.average(time_residuals)
sigma = np.std(time_residuals)
#normalverteilung mit diesem Mittelwert und Standardabweichung
y = norm.pdf(bins, mu, sigma)
ax2.plot(bins, y, 'r--')
# Anzeigen der Plots
plt.tight_layout()
plt.show()

(e) Berechnen und interpretieren Sie die R2 Werte für die beiden Fits. 

In [6]:
# Berechnung der Bestimmtheitsmaße (R^2) für beide Regressionen
r_squared_years = years_model.score(X_years, absent_hours)
r_squared_age = age_model.score(X_age, absent_hours)


print(f'R^2 Jahre im Betrieb: {r_squared_years}')
print(f'R^2 Alter: {r_squared_age}')

(f) Was würden Sie der Firma raten?

## 4

Eine grosse Befragung der Bevölkerung wurde durchgeführt und ihre Resultate sind in der Datei Umfragedaten_v1_an.csv 
(Quelle: allgemeine Bevölkerungsumfrage der Sozialwissenschaften 2014 (ALLBUS 2014)). 
Als Data Scientist sollen Sie klären, ob das Rauchverhalten der Bevölkerung vom Einkommen abhängt

(a) Berechnen Sie ein logistisches Model

In [10]:
from sklearn.linear_model import LogisticRegression

umf = pd.read_csv('data/Umfragedaten_v1_an.csv')
umf = umf.dropna(subset=['NETTO', 'RAUCH'])
# X = umf['RAUCH']
# y = umf['NETTO']
# model = LogisticRegression.fit()
umf.head()